# Deriving expected moment dynamics
$$
\def\n{\mathbf{n}}
\def\x{\mathbf{x}}
\def\N{\mathbb{\mathbb{N}}}
\def\X{\mathbb{X}}
\def\NX{\mathbb{\N_0^\X}}
\def\C{\mathcal{C}}
\def\Jc{\mathcal{J}_c}
\def\DM{\Delta M_{c,j}}
\newcommand\diff{\mathop{}\!\mathrm{d}}
\def\Xc{\mathbf{X}_c}
\def\Yc{\mathbf{Y}_c}
\newcommand{\muset}[1]{\dot{\{}#1\dot{\}}}
$$

In [1]:
# initialize sympy printing (for latex output)
from sympy import init_printing
init_printing()

# import functions and classes for compartment models
from compartments import *
from compartments_display import *

## Specifying the reaction network

As in the previous HOWTO, first specify a reaction network.
We use again the nested birth-death process example.

In [2]:
D = 1 # number of species

y = ContentVar('y')
x = ContentVar('x')

# Intake
transition_I = Transition(EmptySet(), Compartment(y), name='I')
k_I = Constant('k_I')
g_I = 1
pi_I = pi_c_poisson(
    Symbol("\pi_I", positive=True),
    y[0],
    Symbol("\lambda", positive=True))

# Exit
transition_E = Transition(Compartment(x), EmptySet(), name='E')
k_E = Constant('k_E')
g_E = 1
pi_E = pi_c_identity()

# birth
transition_b = Transition(Compartment(x), Compartment(x + ContentChange(1)), name='b')
k_b = Constant('k_b')
g_b = 1
pi_b = pi_c_identity()

# death
transition_d = Transition(Compartment(x), Compartment(x + ContentChange(-1)), name='d')
k_d = Constant('k_d')
g_d = x[0] # TODO x should be enough here, in case D=1.
pi_d = pi_c_identity()

transitions = [
    (transition_I, k_I, g_I, pi_I),
    (transition_E, k_E, g_E, pi_E),
    (transition_b, k_b, g_b, pi_b),
    (transition_d, k_d, g_d, pi_d)
]

display_transitions(transitions)

EmptySet() ---> [y])

[x] ---> EmptySet())

[x] ---> [(1,) + x])

[x] ---> [(-1,) + x])

## Deriving expected moment dynamics

Next we specify the moments we want to derive, for example, $N$, $N^2$, $M^1$, and $(M^1)^2$.

To denote a particular moment, use the `Moment` class. It takes as constructor arguments the integer exponents in each species.

In [4]:
moments = {
    Moment(0),
    Moment(0)**2,
    Moment(1),
    Moment(1)**2
}
display(moments)

⎧                 2                      2   ⎫
⎨Moment(0), Moment (0), Moment(1), Moment (1)⎬
⎩                                            ⎭

Then call the function `compute_moment_evolutions()`, giving the reaction network, the desired moment expressions, and the number of species.

In [6]:
(evolutions, missing) = compute_moment_evolutions(transitions, moments, D)

The result is a pair `(evolutions, missing)`.

`evolutions` are the expressions for the expectation of the desired moment expressions.
`evolutions` is a list of pairs `(fM, dfMdt)`, where each pair consists of the desired moment expression, and the derived expression for its derivative by $t$.

`missing` is a set of moment expressions that are reuired in the derivatives, but are not known yet (not listed in desired `moments`).

To display the derived evolutions, use the `display_expected_moment_evolutions()` function.

In [7]:
display_expected_moment_evolutions(evolutions, D)

d                                             
──(Expectation(N)) = -k_E⋅Expectation(N) + k_I
dt                                            

d ⎛           ⎛      2   ⎞⎞          2                                        
──⎝Expectation⎝Moment (1)⎠⎠ = \lambda ⋅k_I + 2⋅\lambda⋅k_I⋅Expectation(Moment(
dt                                                                            

                                     ⎛      2   ⎞                             
1)) + \lambda⋅k_I - 2⋅k_E⋅Expectation⎝Moment (1)⎠ + k_E⋅Expectation(Moment(2))
                                                                              

                                                                              
 + k_b⋅Expectation(N) + 2⋅k_b⋅Expectation(N⋅Moment(1)) + k_d⋅Expectation(Momen
                                                                              

                         ⎛      2   ⎞
t(1)) - 2⋅k_d⋅Expectation⎝Moment (1)⎠
                                     

d                                                                             
──(Expectation(Moment(1))) = \lambda⋅k_I - k_E⋅Expectation(Moment(1)) + k_b⋅Ex
dt                                                                            

                                         
pectation(N) - k_d⋅Expectation(Moment(1))
                                         

d ⎛           ⎛ 2⎞⎞                                         ⎛ 2⎞              
──⎝Expectation⎝N ⎠⎠ = k_E⋅Expectation(N) - 2⋅k_E⋅Expectation⎝N ⎠ + 2⋅k_I⋅Expec
dt                                                                            

               
tation(N) + k_I
               

The missing moment expressions occuring in these equations are `missing`.

In [8]:
missing

{Moment(0)⋅Moment(1), Moment(2)}

Now, we can iterate the process to also get these moments.

In [10]:
(evolutions2, missing2) = compute_moment_evolutions(transitions, missing, D, provided=moments)
display_expected_moment_evolutions(evolutions2, D)

d                                                                             
──(Expectation(N⋅Moment(1))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I - 2⋅k_
dt                                                                            

                                                                              
E⋅Expectation(N⋅Moment(1)) + k_E⋅Expectation(Moment(1)) + k_I⋅Expectation(Mome
                                                                              

                        ⎛ 2⎞                               
nt(1)) + k_b⋅Expectation⎝N ⎠ - k_d⋅Expectation(N⋅Moment(1))
                                                           

d                                   2                                         
──(Expectation(Moment(2))) = \lambda ⋅k_I + \lambda⋅k_I - k_E⋅Expectation(Mome
dt                                                                            

                                                                              
nt(2)) + k_b⋅Expectation(N) + 2⋅k_b⋅Expectation(Moment(1)) + k_d⋅Expectation(M
                                                                              

                                        
oment(1)) - 2⋅k_d⋅Expectation(Moment(2))
                                        

In this example, the system is closed, as there are no more missing moment expressions.

In [13]:
missing2

Otherwise, the above process can be iterated until convergence or up to the desired order of moments (and applying a moment closure).